In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import time
from selenium import webdriver
import glob
import os
import csv

https://www.koreabaseball.com/Record/Player/HitterBasic/Basic1.aspx

## 1) hitter

In [3]:
api_delay_term = 5

In [4]:
# crawling_hitter_basic_stats
def crawling_hitter_basic(season_id, team_id):
    """
    season_id = 0 ~ 34
    team_id = 1 ~ 10
    ------------------------------------------------------------------------------------
    <season_id>
    1982: 1982 ~ 34: 2017
    
    <team_id> ==> It can be different from several season.
    1 : Nexen heroes
    2 : Doosan
    3 : Lotte
    4 : Samsung
    5 : Hanhwa
    6 : KIA
    7 : KT
    8 : LG twins
    9 : NC dinos
    10 : SK wyberns
    """
    
    # connect url
    driver = webdriver.PhantomJS()
    url = "http://www.koreabaseball.com/Record/Player/HitterBasic/Basic1.aspx"
    driver.get(url)
    
    # click season
    driver.find_element_by_css_selector('#cphContents_cphContents_cphContents_ddlSeason_ddlSeason').\
            find_elements_by_css_selector('option')[season_id].click()
    time.sleep(api_delay_term)
    
    # click team
    driver.find_element_by_css_selector('#cphContents_cphContents_cphContents_ddlTeam_ddlTeam').\
            find_elements_by_css_selector('option')[team_id].click()
    time.sleep(api_delay_term)
    
    # get page number
    page_elements = driver.find_elements_by_css_selector(".paging a")
    page_number = len(page_elements)
    if page_number == 1:
        page_number = page_number
    
    if page_number > 1:
        page_number = page_number -2

    # make empty dataframe
    hitter_basic_df = pd.DataFrame(columns=[
        'rank', 'name', 'team', 'avg', 'g', 'pa', 'ab', 'r', 'h', '2b',
        '3b', 'hr', 'tb', 'rbi', 'sac', 'sf'
    ])
    
    # if having one page
    if page_number == 1:
        elements = driver.find_elements_by_css_selector(".record_result tr")
        elements = elements[1:len(elements)+1]

        for element in elements:
            tmp_dict  = {
                'rank' : element.find_elements_by_css_selector('td')[0].text,
                'name' : element.find_elements_by_css_selector('td')[1].text,
                'team' : element.find_elements_by_css_selector('td')[2].text,
                'avg' : element.find_elements_by_css_selector('td')[3].text,
                'g' : element.find_elements_by_css_selector('td')[4].text,
                'pa' : element.find_elements_by_css_selector('td')[5].text,
                'ab' : element.find_elements_by_css_selector('td')[6].text,
                'r' : element.find_elements_by_css_selector('td')[7].text,
                'h' : element.find_elements_by_css_selector('td')[8].text,
                '2b' : element.find_elements_by_css_selector('td')[9].text,
                '3b' : element.find_elements_by_css_selector('td')[10].text,
                'hr' : element.find_elements_by_css_selector('td')[11].text,
                'tb' : element.find_elements_by_css_selector('td')[12].text,
                'rbi' : element.find_elements_by_css_selector('td')[13].text,
                'sac' : element.find_elements_by_css_selector('td')[14].text,
                'sf' : element.find_elements_by_css_selector('td')[15].text,
            }
            hitter_basic_df.loc[len(hitter_basic_df)] = tmp_dict
    
    # if having other more pages
    if page_number > 1:
        for page in range(1, page_number+1):
            driver.find_element_by_css_selector('#cphContents_cphContents_cphContents_ucPager_btnNo' + str(page)).click()
            time.sleep(api_delay_term)
            
            elements = driver.find_elements_by_css_selector(".record_result tr")
            elements = elements[1:len(elements)+1]
            
            for element in elements:
                tmp_dict  = {
                    'rank' : element.find_elements_by_css_selector('td')[0].text,
                    'name' : element.find_elements_by_css_selector('td')[1].text,
                    'team' : element.find_elements_by_css_selector('td')[2].text,
                    'avg' : element.find_elements_by_css_selector('td')[3].text,
                    'g' : element.find_elements_by_css_selector('td')[4].text,
                    'pa' : element.find_elements_by_css_selector('td')[5].text,
                    'ab' : element.find_elements_by_css_selector('td')[6].text,
                    'r' : element.find_elements_by_css_selector('td')[7].text,
                    'h' : element.find_elements_by_css_selector('td')[8].text,
                    '2b' : element.find_elements_by_css_selector('td')[9].text,
                    '3b' : element.find_elements_by_css_selector('td')[10].text,
                    'hr' : element.find_elements_by_css_selector('td')[11].text,
                    'tb' : element.find_elements_by_css_selector('td')[12].text,
                    'rbi' : element.find_elements_by_css_selector('td')[13].text,
                    'sac' : element.find_elements_by_css_selector('td')[14].text,
                    'sf' : element.find_elements_by_css_selector('td')[15].text,
                }
                hitter_basic_df.loc[len(hitter_basic_df)] = tmp_dict
                
    driver.close()
                
    return hitter_basic_df

In [11]:
# crawling_hitter_detail_stats
def crawling_hitter_detail(season_id, team_id):
    """
    season_id = 0 ~ 34
    team_id = 1 ~ 10
    ------------------------------------------------------------------------------------
    <season_id>
    0 : 1982 ~ 34 : 2016
    
    <team_id> ==> It can be different from several season.
    1 : Nexen heroes
    2 : Doosan
    3 : Lotte
    4 : Samsung
    5 : Hanhwa
    6 : KIA
    7 : KT
    8 : LG twins
    9 : NC dinos
    10 : SK wyberns
    """
    driver = webdriver.PhantomJS()
    url = "http://www.koreabaseball.com/Record/Player/HitterBasic/Detail1.aspx"
    driver.get(url)
    
    # click season
    driver.find_element_by_css_selector('#cphContents_cphContents_cphContents_ddlSeason_ddlSeason').\
            find_elements_by_css_selector('option')[season_id].click()
    time.sleep(api_delay_term)
    
    # click team
    driver.find_element_by_css_selector('#cphContents_cphContents_cphContents_ddlTeam_ddlTeam').\
            find_elements_by_css_selector('option')[team_id].click()
    time.sleep(api_delay_term)
    
    # get page number
    page_elements = driver.find_elements_by_css_selector(".paging a")
    page_number = len(page_elements)
    if page_number == 1:
        page_number = page_number
    
    if page_number > 1:
        page_number = page_number -2
        
    # make empty dataframe
    hitter_detail_df = pd.DataFrame(columns=[
        'rank', 'name', 'team', 'avg', 'xbh', 'go', 'ao', 'go/ao', 'gw rbi',
        'bb/k', 'p/pa', 'isop', 'xr', 'gpa'
    ])
    
    # if having one page
    if page_number == 1:
        elements = driver.find_elements_by_css_selector(".record_result tr")
        elements = elements[1:len(elements)+1]
    
        for element in elements:
            tmp_dict  = {
                'rank' : element.find_elements_by_css_selector('td')[0].text,
                'name' : element.find_elements_by_css_selector('td')[1].text,
                'team' : element.find_elements_by_css_selector('td')[2].text,
                'avg' : element.find_elements_by_css_selector('td')[3].text,
                'xbh' : element.find_elements_by_css_selector('td')[4].text,
                'go' : element.find_elements_by_css_selector('td')[5].text,
                'ao' : element.find_elements_by_css_selector('td')[6].text,
                'go/ao' : element.find_elements_by_css_selector('td')[7].text,
                'gw rbi' : element.find_elements_by_css_selector('td')[8].text,
                'bb/k' : element.find_elements_by_css_selector('td')[9].text,
                'p/pa' : element.find_elements_by_css_selector('td')[10].text,
                'isop' : element.find_elements_by_css_selector('td')[11].text,
                'xr' : element.find_elements_by_css_selector('td')[12].text,
                'gpa' : element.find_elements_by_css_selector('td')[13].text,
            }
            hitter_detail_df.loc[len(hitter_detail_df)] = tmp_dict
        
    # if having other more pages
    if page_number > 1:
        for page in range(1, page_number+1):
            driver.find_element_by_css_selector('#cphContents_cphContents_cphContents_ucPager_btnNo' + str(page)).click()
            time.sleep(api_delay_term)
            
            elements = driver.find_elements_by_css_selector(".record_result tr")
            elements = elements[1:len(elements)+1]
            
            for element in elements:
                tmp_dict  = {
                    'rank' : element.find_elements_by_css_selector('td')[0].text,
                    'name' : element.find_elements_by_css_selector('td')[1].text,
                    'team' : element.find_elements_by_css_selector('td')[2].text,
                    'avg' : element.find_elements_by_css_selector('td')[3].text,
                    'xbh' : element.find_elements_by_css_selector('td')[4].text,
                    'go' : element.find_elements_by_css_selector('td')[5].text,
                    'ao' : element.find_elements_by_css_selector('td')[6].text,
                    'go/ao' : element.find_elements_by_css_selector('td')[7].text,
                    'gw rbi' : element.find_elements_by_css_selector('td')[8].text,
                    'bb/k' : element.find_elements_by_css_selector('td')[9].text,
                    'p/pa' : element.find_elements_by_css_selector('td')[10].text,
                    'isop' : element.find_elements_by_css_selector('td')[11].text,
                    'xr' : element.find_elements_by_css_selector('td')[12].text,
                    'gpa' : element.find_elements_by_css_selector('td')[13].text,
                }
                hitter_detail_df.loc[len(hitter_detail_df)] = tmp_dict
                
    driver.close()
    
    return hitter_detail_df

In [13]:
df0 = crawling_hitter_basic(34, 1)
df1 = crawling_hitter_detail(34, 1)
df = pd.concat([df0, df1], axis=1)

In [14]:
for team in range(2, 10+1):
    print(team)
    df0 = crawling_hitter_basic(34, team)
    df1 = crawling_hitter_detail(34, team)
    df2 = pd.concat([df0, df1], axis=1)
    
    df = pd.concat([df, df2], axis=0, ignore_index=True)

2
3
4
5
6
7
8
9
10


In [25]:
df.tail()

,rank,name,team,avg,g,pa,ab,r,h,2b,...,xbh,go,ao,go/ao,gw rbi,bb/k,p/pa,isop,xr,gpa
373,25,이정담,SK,-,1,0,0,0,0,0,...,0,0,0,-,0,-,-,-,0.0,0.000
374,25,전유수,SK,-,3,0,0,0,0,0,...,0,0,0,-,0,-,-,-,0.0,0.000
375,25,정영일,SK,-,2,0,0,0,0,0,...,0,0,0,-,0,-,-,-,0.0,0.000
376,25,조한욱,SK,-,1,0,0,0,0,0,...,0,0,0,-,0,-,-,-,0.0,0.000
377,25,채병용,SK,-,2,0,0,0,0,0,...,0,0,0,-,0,-,-,-,0.0,0.000


In [24]:
df.to_csv('./hitter.csv', encoding='utf-8')

## 2) pitcher

In [27]:
# crawling_pitcher_basic
def crawling_pitcher_basic(season_id, team_id):
    """
    season_id = 0 ~ 34
    team_id = 1 ~ 10
    ------------------------------------------------------------------------------------
    <season_id>
    0 : 1982 ~ 34 : 2016
    
    <team_id> ==> It can be different from several season.
    1 : Nexen heroes
    2 : Doosan
    3 : Lotte
    4 : Samsung
    5 : Hanhwa
    6 : KIA
    7 : KT
    8 : LG twins
    9 : NC dinos
    10 : SK wyberns
    """
    driver = webdriver.PhantomJS()
    url = "http://www.koreabaseball.com/Record/Player/PitcherBasic/Basic1.aspx"
    driver.get(url)
    
    # click season
    driver.find_element_by_css_selector('#cphContents_cphContents_cphContents_ddlSeason_ddlSeason').\
            find_elements_by_css_selector('option')[season_id].click()
    time.sleep(api_delay_term)
    
    # click team
    driver.find_element_by_css_selector('#cphContents_cphContents_cphContents_ddlTeam_ddlTeam').\
            find_elements_by_css_selector('option')[team_id].click()
    time.sleep(api_delay_term)
    
    # get page number
    page_elements = driver.find_elements_by_css_selector(".paging a")
    page_number = len(page_elements)
    if page_number == 1:
        page_number = page_number
    
    if page_number > 1:
        page_number = page_number -2
        
    # make empty dataframe
    pitcher_basic_df = pd.DataFrame(columns=[
        'rank', 'name', 'team', 'ERA', 'G', 'W', 'L', 'SV', 'HLD', 'WPCT',
        'IP', 'H', 'HR', 'BB', 'HBP', 'SO', 'R', 'ER', 'WHIP'
    ])
    
    # if having one page
    if page_number == 1:
        elements = driver.find_elements_by_css_selector(".record_result tr")
        elements = elements[1:len(elements)+1]
        
        for element in elements:
            tmp_dict  = {
                'rank' : element.find_elements_by_css_selector('td')[0].text,
                'name' : element.find_elements_by_css_selector('td')[1].text,
                'team' : element.find_elements_by_css_selector('td')[2].text,
                'ERA' : element.find_elements_by_css_selector('td')[3].text,
                'G' : element.find_elements_by_css_selector('td')[4].text,
                'W' : element.find_elements_by_css_selector('td')[5].text,
                'L' : element.find_elements_by_css_selector('td')[6].text,
                'SV' : element.find_elements_by_css_selector('td')[7].text,
                'HLD' : element.find_elements_by_css_selector('td')[8].text,
                'WPCT' : element.find_elements_by_css_selector('td')[9].text,
                'IP' : element.find_elements_by_css_selector('td')[10].text,
                'H' : element.find_elements_by_css_selector('td')[11].text,
                'HR' : element.find_elements_by_css_selector('td')[12].text,
                'BB' : element.find_elements_by_css_selector('td')[13].text,
                'HBP' : element.find_elements_by_css_selector('td')[14].text,
                'SO' : element.find_elements_by_css_selector('td')[15].text,
                'R' : element.find_elements_by_css_selector('td')[16].text,
                'ER' : element.find_elements_by_css_selector('td')[17].text,
                'WHIP' : element.find_elements_by_css_selector('td')[18].text,
            }
            pitcher_basic_df.loc[len(pitcher_basic_df)] = tmp_dict
        
    # if having other more pages
    if page_number > 1:
        for page in range(1, page_number+1):
            driver.find_element_by_css_selector('#cphContents_cphContents_cphContents_ucPager_btnNo' + str(page)).click()
            time.sleep(api_delay_term)
            
            elements = driver.find_elements_by_css_selector(".record_result tr")
            elements = elements[1:len(elements)+1]
            
            for element in elements:
                tmp_dict  = {
                    'rank' : element.find_elements_by_css_selector('td')[0].text,
                    'name' : element.find_elements_by_css_selector('td')[1].text,
                    'team' : element.find_elements_by_css_selector('td')[2].text,
                    'ERA' : element.find_elements_by_css_selector('td')[3].text,
                    'G' : element.find_elements_by_css_selector('td')[4].text,
                    'W' : element.find_elements_by_css_selector('td')[5].text,
                    'L' : element.find_elements_by_css_selector('td')[6].text,
                    'SV' : element.find_elements_by_css_selector('td')[7].text,
                    'HLD' : element.find_elements_by_css_selector('td')[8].text,
                    'WPCT' : element.find_elements_by_css_selector('td')[9].text,
                    'IP' : element.find_elements_by_css_selector('td')[10].text,
                    'H' : element.find_elements_by_css_selector('td')[11].text,
                    'HR' : element.find_elements_by_css_selector('td')[12].text,
                    'BB' : element.find_elements_by_css_selector('td')[13].text,
                    'HBP' : element.find_elements_by_css_selector('td')[14].text,
                    'SO' : element.find_elements_by_css_selector('td')[15].text,
                    'R' : element.find_elements_by_css_selector('td')[16].text,
                    'ER' : element.find_elements_by_css_selector('td')[17].text,
                    'WHIP' : element.find_elements_by_css_selector('td')[18].text,
                }
                pitcher_basic_df.loc[len(pitcher_basic_df)] = tmp_dict
    
    driver.close()
    return pitcher_basic_df

In [28]:
# crawling_pitcher_detail
def crawling_pitcher_detail(season_id, team_id):
    """
    season_id = 0 ~ 34
    team_id = 1 ~ 10
    ------------------------------------------------------------------------------------
    <season_id>
    0 : 1982 ~ 34 : 2016
    
    <team_id> ==> It can be different from several season.
    1 : Nexen heroes
    2 : Doosan
    3 : Lotte
    4 : Samsung
    5 : Hanhwa
    6 : KIA
    7 : KT
    8 : LG twins
    9 : NC dinos
    10 : SK wyberns
    """
    driver = webdriver.PhantomJS()
    url = "http://www.koreabaseball.com/Record/Player/PitcherBasic/Detail1.aspx"
    driver.get(url)
    
    # click season
    driver.find_element_by_css_selector('#cphContents_cphContents_cphContents_ddlSeason_ddlSeason').\
            find_elements_by_css_selector('option')[season_id].click()
    time.sleep(api_delay_term)
    
    # click team
    driver.find_element_by_css_selector('#cphContents_cphContents_cphContents_ddlTeam_ddlTeam').\
            find_elements_by_css_selector('option')[team_id].click()
    time.sleep(api_delay_term)
    
    # get page number
    page_elements = driver.find_elements_by_css_selector(".paging a")
    page_number = len(page_elements)
    if page_number == 1:
        page_number = page_number
    
    if page_number > 1:
        page_number = page_number -2
        
    # make empty dataframe
    pitcher_detail_df = pd.DataFrame(columns=[
        'rank', 'name', 'team', 'ERA', 'GS', 'Wgs', 'Wgr', 'GF', 'SVO', 'TS',
        'GDP', 'GO', 'AO', 'GO/AO'
    ])
    
    # if having one page
    if page_number == 1:
        elements = driver.find_elements_by_css_selector(".record_result tr")
        elements = elements[1:len(elements)+1]
        
        for element in elements:
            tmp_dict  = {
                'rank' : element.find_elements_by_css_selector('td')[0].text,
                'name' : element.find_elements_by_css_selector('td')[1].text,
                'team' : element.find_elements_by_css_selector('td')[2].text,
                'ERA' : element.find_elements_by_css_selector('td')[3].text,
                'GS' : element.find_elements_by_css_selector('td')[4].text,
                'Wgs' : element.find_elements_by_css_selector('td')[5].text,
                'Wgr' : element.find_elements_by_css_selector('td')[6].text,
                'GF' : element.find_elements_by_css_selector('td')[7].text,
                'SVO' : element.find_elements_by_css_selector('td')[8].text,
                'TS' : element.find_elements_by_css_selector('td')[9].text,
                'GDP' : element.find_elements_by_css_selector('td')[10].text,
                'GO' : element.find_elements_by_css_selector('td')[11].text,
                'AO' : element.find_elements_by_css_selector('td')[12].text,
                'GO/AO' : element.find_elements_by_css_selector('td')[13].text,
            }
            pitcher_detail_df.loc[len(pitcher_detail_df)] = tmp_dict
        
    # if having other more pages
    if page_number > 1:
        for page in range(1, page_number+1):
            driver.find_element_by_css_selector('#cphContents_cphContents_cphContents_ucPager_btnNo' + str(page)).click()
            time.sleep(api_delay_term)
            
            elements = driver.find_elements_by_css_selector(".record_result tr")
            elements = elements[1:len(elements)+1]
            
            for element in elements:
                tmp_dict  = {
                    'rank' : element.find_elements_by_css_selector('td')[0].text,
                    'name' : element.find_elements_by_css_selector('td')[1].text,
                    'team' : element.find_elements_by_css_selector('td')[2].text,
                    'ERA' : element.find_elements_by_css_selector('td')[3].text,
                    'GS' : element.find_elements_by_css_selector('td')[4].text,
                    'Wgs' : element.find_elements_by_css_selector('td')[5].text,
                    'Wgr' : element.find_elements_by_css_selector('td')[6].text,
                    'GF' : element.find_elements_by_css_selector('td')[7].text,
                    'SVO' : element.find_elements_by_css_selector('td')[8].text,
                    'TS' : element.find_elements_by_css_selector('td')[9].text,
                    'GDP' : element.find_elements_by_css_selector('td')[10].text,
                    'GO' : element.find_elements_by_css_selector('td')[11].text,
                    'AO' : element.find_elements_by_css_selector('td')[12].text,
                    'GO/AO' : element.find_elements_by_css_selector('td')[13].text,
                }
                pitcher_detail_df.loc[len(pitcher_detail_df)] = tmp_dict
    
    driver.close()
    return pitcher_detail_df

In [29]:
df0 = crawling_pitcher_basic(34, 1)
df1 = crawling_pitcher_detail(34, 1)
df = pd.concat([df0, df1], axis=1)

In [30]:
for team in range(2, 10+1):
    print(team)
    df0 = crawling_pitcher_basic(34, team)
    df1 = crawling_pitcher_detail(34, team)
    df2 = pd.concat([df0, df1], axis=1)
    
    df = pd.concat([df, df2], axis=0, ignore_index=True)

2
3
4
5
6
7
8
9
10


In [32]:
df.to_csv('./pitcher.csv', encoding='utf-8')

In [32]:
# crawling_defense
def crawling_defense(season_id, team_id):
    """
    season_id = 0 ~ 14
    team_id = 1 ~ 10
    ------------------------------------------------------------------------------------
    <season_id>
    0 : 2002 ~ 14 : 2016
    
    <team_id> ==> It can be different from several season.
    1 : Nexen heroes
    2 : Doosan
    3 : Lotte
    4 : Samsung
    5 : Hanhwa
    6 : KIA
    7 : KT
    8 : LG twins
    9 : NC dinos
    10 : SK wyberns
    """
    driver = webdriver.Firefox()
    url = "http://www.koreabaseball.com/Record/Player/Defense/Basic.aspx"
    driver.get(url)
    
    # click season
    driver.find_element_by_css_selector('#cphContents_cphContents_cphContents_ddlSeason_ddlSeason').\
            find_elements_by_css_selector('option')[season_id].click()
    time.sleep(api_delay_term)
    
    # click team
    driver.find_element_by_css_selector('#cphContents_cphContents_cphContents_ddlTeam_ddlTeam').\
            find_elements_by_css_selector('option')[team_id].click()
    time.sleep(api_delay_term)
    
    # get page number
    page_elements = driver.find_elements_by_css_selector(".paging a")
    page_number = len(page_elements)
    if page_number == 1:
        page_number = page_number
    
    if page_number > 1:
        page_number = page_number -2
        
    # make empty dataframe
    defense_df = pd.DataFrame(columns=[
        'rank', 'name', 'team', 'POS', 'G', 'GS', 'IP', 'E', 'PKO', 'PO',
        'A', 'DP', 'FPCT', 'PB', 'SB', 'CS', 'CS%'
    ])
    
    # if having one page
    if page_number == 1:
        elements = driver.find_elements_by_css_selector(".record_result tr")
        elements = elements[1:len(elements)+1]
        
        for element in elements:
            tmp_dict  = {
                'rank' : element.find_elements_by_css_selector('td')[0].text,
                'name' : element.find_elements_by_css_selector('td')[1].text,
                'team' : element.find_elements_by_css_selector('td')[2].text,
                'POS' : element.find_elements_by_css_selector('td')[3].text,
                'G' : element.find_elements_by_css_selector('td')[4].text,
                'GS' : element.find_elements_by_css_selector('td')[5].text,
                'IP' : element.find_elements_by_css_selector('td')[6].text,
                'E' : element.find_elements_by_css_selector('td')[7].text,
                'PKO' : element.find_elements_by_css_selector('td')[8].text,
                'PO' : element.find_elements_by_css_selector('td')[9].text,
                'A' : element.find_elements_by_css_selector('td')[10].text,
                'DP' : element.find_elements_by_css_selector('td')[11].text,
                'FPCT' : element.find_elements_by_css_selector('td')[12].text,
                'PB' : element.find_elements_by_css_selector('td')[13].text,
                'SB' : element.find_elements_by_css_selector('td')[14].text,
                'CS' : element.find_elements_by_css_selector('td')[15].text,
                'CS%' : element.find_elements_by_css_selector('td')[16].text,
            }
            defense_df.loc[len(defense_df)] = tmp_dict
        
    # if having other more pages
    if page_number > 1:
        for page in range(1, page_number+1):
            driver.find_element_by_css_selector('#cphContents_cphContents_cphContents_ucPager_btnNo' + str(page)).click()
            time.sleep(api_delay_term)
            
            elements = driver.find_elements_by_css_selector(".record_result tr")
            elements = elements[1:len(elements)+1]
            
            for element in elements:
                tmp_dict  = {
                    'rank' : element.find_elements_by_css_selector('td')[0].text,
                    'name' : element.find_elements_by_css_selector('td')[1].text,
                    'team' : element.find_elements_by_css_selector('td')[2].text,
                    'POS' : element.find_elements_by_css_selector('td')[3].text,
                    'G' : element.find_elements_by_css_selector('td')[4].text,
                    'GS' : element.find_elements_by_css_selector('td')[5].text,
                    'IP' : element.find_elements_by_css_selector('td')[6].text,
                    'E' : element.find_elements_by_css_selector('td')[7].text,
                    'PKO' : element.find_elements_by_css_selector('td')[8].text,
                    'PO' : element.find_elements_by_css_selector('td')[9].text,
                    'A' : element.find_elements_by_css_selector('td')[10].text,
                    'DP' : element.find_elements_by_css_selector('td')[11].text,
                    'FPCT' : element.find_elements_by_css_selector('td')[12].text,
                    'PB' : element.find_elements_by_css_selector('td')[13].text,
                    'SB' : element.find_elements_by_css_selector('td')[14].text,
                    'CS' : element.find_elements_by_css_selector('td')[15].text,
                    'CS%' : element.find_elements_by_css_selector('td')[16].text,
                }
                defense_df.loc[len(defense_df)] = tmp_dict

    return defense_df